# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 8 2022 1:09PM,252639,12,HH-38701,Hush Hush,Released
1,Dec 8 2022 1:09PM,252639,12,HH-38702,Hush Hush,Released
2,Dec 8 2022 1:09PM,252639,12,HH-38703,Hush Hush,Released
3,Dec 8 2022 1:09PM,252639,12,HH-38704,Hush Hush,Released
4,Dec 8 2022 1:00PM,252637,21,418543299,"NBTY Global, Inc.",Released
5,Dec 8 2022 12:55PM,252636,21,418543315,"NBTY Global, Inc.",Released
6,Dec 8 2022 12:53PM,252635,10,Stone-8877604,Acute Outpatient Solutions,Released
7,Dec 8 2022 12:53PM,252634,10,MSP216577,"Methapharm, Inc.",Released
8,Dec 8 2022 12:53PM,252634,10,MSP216578,"Methapharm, Inc.",Released
9,Dec 8 2022 12:53PM,252634,10,MSP216579,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
68,252634,Released,6
69,252635,Released,1
70,252636,Released,1
71,252637,Released,1
72,252639,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
252634,NaN,NaN,NaN,6.0
252635,NaN,NaN,NaN,1.0
252636,NaN,NaN,NaN,1.0
252637,NaN,NaN,NaN,1.0
252639,NaN,NaN,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
252523,0.0,0.0,0.0,1.0
252525,0.0,0.0,0.0,1.0
252526,0.0,0.0,0.0,1.0
252528,0.0,0.0,0.0,1.0
252531,0.0,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
252523,0.0,0,0,1
252525,0.0,0,0,1
252526,0.0,0,0,1
252528,0.0,0,0,1
252531,0.0,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,252523,0.0,0,0,1
1,252525,0.0,0,0,1
2,252526,0.0,0,0,1
3,252528,0.0,0,0,1
4,252531,0.0,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,252523,0.0,,,1
1,252525,0.0,,,1
2,252526,0.0,,,1
3,252528,0.0,,,1
4,252531,0.0,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 8 2022 1:09PM,252639,12,Hush Hush
4,Dec 8 2022 1:00PM,252637,21,"NBTY Global, Inc."
5,Dec 8 2022 12:55PM,252636,21,"NBTY Global, Inc."
6,Dec 8 2022 12:53PM,252635,10,Acute Outpatient Solutions
7,Dec 8 2022 12:53PM,252634,10,"Methapharm, Inc."
13,Dec 8 2022 12:52PM,252633,19,"GUSA Granules USA, Inc."
14,Dec 8 2022 12:48PM,252607,15,"Virtus Pharmaceuticals, LLC"
15,Dec 8 2022 12:48PM,252632,10,Bio-PRF
21,Dec 8 2022 12:48PM,252630,10,ISDIN Corporation
26,Dec 8 2022 12:47PM,252629,15,"Mizner Bioscience, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Executing,Released
0,Dec 8 2022 1:09PM,252639,12,Hush Hush,0.0,,,4
1,Dec 8 2022 1:00PM,252637,21,"NBTY Global, Inc.",0.0,,,1
2,Dec 8 2022 12:55PM,252636,21,"NBTY Global, Inc.",0.0,,,1
3,Dec 8 2022 12:53PM,252635,10,Acute Outpatient Solutions,0.0,,,1
4,Dec 8 2022 12:53PM,252634,10,"Methapharm, Inc.",0.0,,,6
5,Dec 8 2022 12:52PM,252633,19,"GUSA Granules USA, Inc.",0.0,,,1
6,Dec 8 2022 12:48PM,252607,15,"Virtus Pharmaceuticals, LLC",0.0,,,1
7,Dec 8 2022 12:48PM,252632,10,Bio-PRF,0.0,,,6
8,Dec 8 2022 12:48PM,252630,10,ISDIN Corporation,0.0,,,5
9,Dec 8 2022 12:47PM,252629,15,"Mizner Bioscience, LLC",0.0,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 8 2022 1:09PM,252639,12,Hush Hush,4,,
1,Dec 8 2022 1:00PM,252637,21,"NBTY Global, Inc.",1,,
2,Dec 8 2022 12:55PM,252636,21,"NBTY Global, Inc.",1,,
3,Dec 8 2022 12:53PM,252635,10,Acute Outpatient Solutions,1,,
4,Dec 8 2022 12:53PM,252634,10,"Methapharm, Inc.",6,,
5,Dec 8 2022 12:52PM,252633,19,"GUSA Granules USA, Inc.",1,,
6,Dec 8 2022 12:48PM,252607,15,"Virtus Pharmaceuticals, LLC",1,,
7,Dec 8 2022 12:48PM,252632,10,Bio-PRF,6,,
8,Dec 8 2022 12:48PM,252630,10,ISDIN Corporation,5,,
9,Dec 8 2022 12:47PM,252629,15,"Mizner Bioscience, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 8 2022 1:09PM,252639,12,Hush Hush,4,,
1,Dec 8 2022 1:00PM,252637,21,"NBTY Global, Inc.",1,,
2,Dec 8 2022 12:55PM,252636,21,"NBTY Global, Inc.",1,,
3,Dec 8 2022 12:53PM,252635,10,Acute Outpatient Solutions,1,,
4,Dec 8 2022 12:53PM,252634,10,"Methapharm, Inc.",6,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 8 2022 1:09PM,252639,12,Hush Hush,4.0,NaN,NaN
1,Dec 8 2022 1:00PM,252637,21,"NBTY Global, Inc.",1.0,NaN,NaN
2,Dec 8 2022 12:55PM,252636,21,"NBTY Global, Inc.",1.0,NaN,NaN
3,Dec 8 2022 12:53PM,252635,10,Acute Outpatient Solutions,1.0,NaN,NaN
4,Dec 8 2022 12:53PM,252634,10,"Methapharm, Inc.",6.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 8 2022 1:09PM,252639,12,Hush Hush,4.0,0.0,0.0
1,Dec 8 2022 1:00PM,252637,21,"NBTY Global, Inc.",1.0,0.0,0.0
2,Dec 8 2022 12:55PM,252636,21,"NBTY Global, Inc.",1.0,0.0,0.0
3,Dec 8 2022 12:53PM,252635,10,Acute Outpatient Solutions,1.0,0.0,0.0
4,Dec 8 2022 12:53PM,252634,10,"Methapharm, Inc.",6.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,6314988,185.0,19.0,1.0
12,505226,5.0,0.0,0.0
15,1263002,12.0,32.0,43.0
16,505159,2.0,0.0,0.0
19,2020719,40.0,14.0,11.0
20,505172,2.0,0.0,2.0
21,4799075,16.0,3.0,0.0
22,252583,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,6314988,185.0,19.0,1.0
1,12,505226,5.0,0.0,0.0
2,15,1263002,12.0,32.0,43.0
3,16,505159,2.0,0.0,0.0
4,19,2020719,40.0,14.0,11.0
5,20,505172,2.0,0.0,2.0
6,21,4799075,16.0,3.0,0.0
7,22,252583,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,185.0,19.0,1.0
1,12,5.0,0.0,0.0
2,15,12.0,32.0,43.0
3,16,2.0,0.0,0.0
4,19,40.0,14.0,11.0
5,20,2.0,0.0,2.0
6,21,16.0,3.0,0.0
7,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,185.0
1,12,Released,5.0
2,15,Released,12.0
3,16,Released,2.0
4,19,Released,40.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,1.0,0.0,43.0,0.0,11.0,2.0,0.0,0.0
Executing,19.0,0.0,32.0,0.0,14.0,0.0,3.0,0.0
Released,185.0,5.0,12.0,2.0,40.0,2.0,16.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,1.0,0.0,43.0,0.0,11.0,2.0,0.0,0.0
1,Executing,19.0,0.0,32.0,0.0,14.0,0.0,3.0,0.0
2,Released,185.0,5.0,12.0,2.0,40.0,2.0,16.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,1.0,0.0,43.0,0.0,11.0,2.0,0.0,0.0
1,Executing,19.0,0.0,32.0,0.0,14.0,0.0,3.0,0.0
2,Released,185.0,5.0,12.0,2.0,40.0,2.0,16.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()